In [1]:
''' 
for the convinience, changing type of flower to ingteger
setosa      0
versicolor  1 
virginica   2

'''

import numpy as np
import pandas as pd

# col_names = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'type']
# data = pd.read_csv("./iris.data", skiprows=1, header=None, names=col_names)
data = pd.read_csv("./iris.data")

data.head(10)

,sepal.lenth,sepal.width,petal.lenth,petal.width,type
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0
5,5.4,3.9,1.7,0.4,0
6,4.6,3.4,1.4,0.3,0
7,5.0,3.4,1.5,0.2,0
8,4.4,2.9,1.4,0.2,0
9,4.9,3.1,1.5,0.1,0


Build Node class

In [2]:
class Node():
    def __init__(self, feature_index=None, threshold=None, left=None, right=None, info_gain=None, value=None):
        ''' constructor 
        feature index : sepal and petal
            threshold : best split data of feature
                left  : left node
                right : node
                info_gain = information gain
        '''      
        # for decision node
        self.feature_index = feature_index 
        self.threshold = threshold
        self.left = left 
        self.right = right
        self.info_gain = info_gain
        
        # for leaf node
        self.value = value

Tree Class

마지막으로 가지치기의 비용함수(cost function)를 살펴보겠습니다. 의사결정나무는 이 비용함수를 최소로 하는 분기를 찾아내도록 학습됩니다. 아래와 같이 정의됩니다.

CC(T)=Err(T)+α×L(T)  
CC(T)=의사결정나무의 비용 복잡도(=오류가 적으면서 terminal node 수가 적은 단순한 모델일 수록 작은 값)

ERR(T)=검증데이터에 대한 오분류율

L(T)=terminal node의 수(구조의 복잡도)

Alpha=ERR(T)와 L(T)를 결합하는 가중치(사용자에 의해 부여됨, 보통 0.01~0.1의 값을 씀)

max_depth: maximum depth of the tree. If we set it to None, the tree will grow until all the leaves are pure or the hyperparameter min_samples_split has been reached.<br>

min_samples_split: indicates the minimum number of observations a sheet must have to continue creating new nodes.<br>

min_information_gain: the minimum amount the Information Gain must increase for the tree to continue growing.<br>

In [3]:
class DecisionTreeClassifier():
    def __init__(self, min_samples_split=2, max_depth=2):
        
        # initialize the root of the tree 
        """ 
                root
            nodeL   nodeR    
        """
        self.root = None
        
        # stopping conditions
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        
    def build_tree(self, dataset, curr_depth=0):
        ''' recursive function to build the tree
            X : attributes(sepal.lenth to petal.width)  
            Y : type (label)
        num_samples : row of feature
        num_features : column of feature
        
        ''' 
        
        X, Y = dataset[:,:-1], dataset[:,-1]
        num_samples, num_features = np.shape(X) 
        print('sample:',num_samples, ' feature:',num_features)
        # split until satisfy stopping conditions
        if num_samples>=self.min_samples_split and curr_depth<=self.max_depth:
            # find the best split
            best_split = self.get_best_split(dataset, num_samples, num_features)
            # check if information gain is positive
            if best_split["info_gain"]>0: #if best split of info gain is zero, the entropy is same(chaotic) 
                # recur left
                left_subtree = self.build_tree(best_split["dataset_left"], curr_depth+1)
                # recur right
                right_subtree = self.build_tree(best_split["dataset_right"], curr_depth+1)
                # return decision node when decision tree still in condition
                return Node(best_split["feature_index"], best_split["threshold"], 
                            left_subtree, right_subtree, best_split["info_gain"])
        
        # compute leaf node
        leaf_value = self.calculate_leaf_value(Y)
        # return leaf node
        return Node(value=leaf_value)
    
    def get_best_split(self, dataset, num_samples, num_features):
        ''' function to find the best split 
            logic:1. take loop and list every feature of index
                  2. list every value of features
                  3. split dataset according to unique of feature values
                  4. dataset value could be null, so if dataset is not null compute impormation gain
                  5. information gain is as better as small, if current gain > max gain , change index
        '''
        
        # dictionary to store the best split
        best_split = {}
        max_info_gain = 0 #-float("inf")
        
        # loop over all the features
        for feature_index in range(num_features):
            feature_values = dataset[:, feature_index]
            possible_thresholds = np.unique(feature_values)
            # loop over all the feature values present in the data
            for threshold in possible_thresholds:
                # get current split
                dataset_left, dataset_right = self.split(dataset, feature_index, threshold)
                # check if childs are not null
                if len(dataset_left)>0 and len(dataset_right)>0:
                    y, left_y, right_y = dataset[:, -1], dataset_left[:, -1], dataset_right[:, -1]
                    # compute information gain
                    curr_info_gain = self.information_gain(y, left_y, right_y, "gini")
                    # update the best split if needed
                    #if current information gain is bigger than max information gain, dataset is less chaotic
                    if curr_info_gain>max_info_gain:
                        best_split["feature_index"] = feature_index
                        best_split["threshold"] = threshold
                        best_split["dataset_left"] = dataset_left
                        best_split["dataset_right"] = dataset_right
                        best_split["info_gain"] = curr_info_gain
                        max_info_gain = curr_info_gain
        print("best_split:",best_split)           
        # return best split
        return best_split
    
    def split(self, dataset, feature_index, threshold):
        ''' function to split the data '''
        
        dataset_left = np.array([row for row in dataset if row[feature_index]<=threshold])
        dataset_right = np.array([row for row in dataset if row[feature_index]>threshold])
        print("threshold:",threshold)
        return dataset_left, dataset_right
    
    def information_gain(self, parent, l_child, r_child, mode="entropy"):
        ''' function to compute information gain '''
        
        weight_l = len(l_child) / len(parent)
        weight_r = len(r_child) / len(parent)
        if mode=="gini":
            gain = self.gini_index(parent) - (weight_l*self.gini_index(l_child) + weight_r*self.gini_index(r_child))
        else:#entropy
            gain = self.entropy(parent) - (weight_l*self.entropy(l_child) + weight_r*self.entropy(r_child))
        return gain
    
    def entropy(self, y):
        ''' function to compute entropy '''
        
        class_labels = np.unique(y)
        entropy = 0
        for cls in class_labels:
            ratio = len(y[y == cls]) / len(y)
            entropy += -ratio * np.log2(ratio)
        return entropy
    
    def gini_index(self, y):
        ''' function to compute gini index '''
        
        class_labels = np.unique(y)
        gini = 0
        for cls in class_labels:
            ratio = len(y[y == cls]) / len(y)
            gini += ratio**2
        return 1 - gini
        
    def calculate_leaf_value(self, Y):
        ''' function to compute leaf node '''
        
        Y = list(Y)
        return max(Y, key=Y.count)
    
    def print_tree(self, tree=None, indent=" "):
        ''' function to print the tree '''
        
        if not tree:
            tree = self.root

        if tree.value is not None:
            print(tree.value)

        else:
            print("X_"+str(tree.feature_index), "<=", tree.threshold, "?", tree.info_gain)
            print("%sleft:" % (indent), end="")
            self.print_tree(tree.left, indent + indent)
            print("%sright:" % (indent), end="")
            self.print_tree(tree.right, indent + indent)
    
    def fit(self, X, Y):
        ''' function to train the tree '''
        
        dataset = np.concatenate((X, Y), axis=1)
        self.root = self.build_tree(dataset)
    
    def predict(self, X):
        ''' function to predict new dataset '''
        
        preditions = [self.make_prediction(x, self.root) for x in X]
        return preditions
    
    def make_prediction(self, x, tree):
        ''' function to predict a single data point '''
        
        if tree.value!=None: return tree.value
        feature_val = x[tree.feature_index]
        if feature_val<=tree.threshold:
            return self.make_prediction(x, tree.left)
        else:
            return self.make_prediction(x, tree.right)

In [4]:
X = data.iloc[:, :-1].values
Y = data.iloc[:, -1].values.reshape(-1,1)
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=.2, random_state=41)

In [5]:
classifier = DecisionTreeClassifier(min_samples_split=3, max_depth=2)
classifier.fit(X_train,Y_train)
classifier.print_tree()

sample: 120  feature: 4
threshold: 4.3
threshold: 4.4
threshold: 4.6
threshold: 4.7
threshold: 4.8
threshold: 4.9
threshold: 5.0
threshold: 5.1
threshold: 5.2
threshold: 5.3
threshold: 5.4
threshold: 5.5
threshold: 5.6
threshold: 5.7
threshold: 5.8
threshold: 5.9
threshold: 6.0
threshold: 6.1
threshold: 6.2
threshold: 6.3
threshold: 6.4
threshold: 6.5
threshold: 6.6
threshold: 6.7
threshold: 6.8
threshold: 6.9
threshold: 7.0
threshold: 7.1
threshold: 7.2
threshold: 7.3
threshold: 7.4
threshold: 7.6
threshold: 7.7
threshold: 7.9
threshold: 2.0
threshold: 2.2
threshold: 2.3
threshold: 2.4
threshold: 2.5
threshold: 2.6
threshold: 2.7
threshold: 2.8
threshold: 2.9
threshold: 3.0
threshold: 3.1
threshold: 3.2
threshold: 3.3
threshold: 3.4
threshold: 3.5
threshold: 3.6
threshold: 3.7
threshold: 3.8
threshold: 3.9
threshold: 4.0
threshold: 4.1
threshold: 4.2
threshold: 4.4
threshold: 1.0
threshold: 1.1
threshold: 1.2
threshold: 1.3
threshold: 1.4
threshold: 1.5
threshold: 1.6
threshold: 1.7
t

KeyError: 'info_gain'

In [ ]:
Y_pred = classifier.predict(X_test) 
from sklearn.metrics import accuracy_score
accuracy_score(Y_test, Y_pred)

0.8666666666666667